# 時中指數

# Load Data 

In [91]:
import pandas as pd
from datetime import datetime, timedelta

df = pd.read_csv('./news_dataset_preprocessed_for_django.csv',sep='|')

In [92]:
df.head(3)

,item_id,date,category,title,content,sentiment,summary,top_key_freq,tokens,tokens_v2,entities,token_pos,link,photo_link
0,_20250327_1,2025-03-27,焦點,台股重挫308點 失守22000關卡,美國總統川普準備徵收汽車關稅，美股主要指數全數下跌，台積電重挫4.09%，台積電台北現股今日...,暫無,暫無,"[('台積電', 4), ('指數', 3), ('月線', 3), ('美國', 2), ...","['美國', '總統', '川普', '準備', '徵收', '汽車', '關稅', '，'...","['美國', '總統', '川普', '汽車', '關稅', '美股', '指數', '台積...","[NerToken(word='美國', ner='GPE', idx=(0, 2)), N...","[('美國', 'Nc'), ('總統', 'Na'), ('川普', 'Nb'), ('準...",https://tw.news.yahoo.com/https://tw.stock.yah...,https://s.yimg.com/ny/api/res/1.2/3qEveVGKp070...
1,_20250327_2,2025-03-27,焦點,賴士葆撞人畫面外流 警被起訴,去年7月間，國民黨立委賴士葆駕車行駛在台北市大安區時，不慎撞傷走在斑馬線上2位女性行人，沒想...,暫無,暫無,"[('台灣', 10), ('賴士葆', 9), ('警員', 9), ('照片', 7),...","['去年', '7月', '間', '，', '國民黨', '立委', '賴士葆', '駕車...","['國民黨', '立委', '賴士葆', '台北市', '大安區', '斑馬線', '女性'...","[NerToken(word='去年7月', ner='DATE', idx=(0, 4))...","[('去年', 'Nd'), ('7月', 'Nd'), ('間', 'Ng'), ('，'...",https://tw.news.yahoo.com/%E8%B3%B4%E5%A3%AB%E...,https://s.yimg.com/ny/api/res/1.2/PPzJSVLdFBTm...
2,_20250327_3,2025-03-27,焦點,墾丁爆非法民宿飆價 署長說重話,台灣祭將於4月3日至5日登場，有民眾在社群平台反映，活動期間發現墾丁地區一間旅館哄抬價格。屏...,暫無,暫無,"[('業者', 8), ('價格', 4), ('旅宿', 4), ('台灣祭', 3), ...","['台灣祭', '將', '於', '4月', '3日', '至', '5日', '登場',...","['台灣祭', '民眾', '社群', '平台', '活動', '期間', '墾丁', '地...","[NerToken(word='台灣祭', ner='EVENT', idx=(0, 3))...","[('台灣祭', 'Na'), ('將', 'D'), ('於', 'P'), ('4月',...",https://tw.news.yahoo.com/%E5%A2%BE%E4%B8%81%E...,https://s.yimg.com/ny/api/res/1.2/E9TwmMfRm0bQ...


In [93]:
df.shape

(216, 14)

# Filter news for selected keywords

In [94]:
import pandas as pd
from datetime import datetime, timedelta
import re

# # Load Data 
# df = pd.read_csv('./news_dataset_preprocessed_for_django.csv',sep='|')
# Step 0: Filter news articles using the following function
# Searching keywords from "content" column
def filter_df_via_content(df, query_keywords, cond, cate, weeks):

    # end date: the date of the latest record of news
    end_date = df.date.max()
    
    # start date
    start_date_delta = (datetime.strptime(end_date, '%Y-%m-%d').date() - timedelta(weeks=weeks)).strftime('%Y-%m-%d')
    start_date_min = df.date.min()
    # set start_date as the larger one from the start_date_delta and start_date_min 開始時間選資料最早時間與周數:兩者較晚者
    start_date = max(start_date_delta,   start_date_min)


    # (1) proceed filtering: a duration of a period of time
    # 期間條件
    period_condition = (df.date >= start_date) & (df.date <= end_date) 
    
    # (2) proceed filtering: news category
    # 新聞類別條件
    if (cate == "全部"):
        condition = period_condition  # "全部"類別不必過濾新聞種類
    else:
        # 過濾category新聞類別條件
        condition = period_condition & (df.category == cate)

    # (3) proceed filtering: and or
    # and or 條件
    if (cond == 'and'):
        # query keywords condition使用者輸入關鍵字條件and
        condition = condition & df.content.apply(lambda text: all((qk in text) for qk in query_keywords)) #寫法:all()
    elif (cond == 'or'):
        # query keywords condition使用者輸入關鍵字條件
        condition = condition & df.content.apply(lambda text: any((qk in text) for qk in query_keywords)) #寫法:any()
    # condiction is a list of True or False boolean value
    df_query = df[condition]

    return df_query


In [95]:
query_keywords = ['川普','美國總統'] 
weeks=4
cond='or'
cate='全部'
df_query = filter_df_via_content(df, query_keywords, cond, cate, weeks)

In [96]:
len(df_query)

41

# Count how many pieces of news containing these keywords計算各類別多少篇文章提到該關鍵字

# Count how many times these keywords were mentioned in each category計算各類別出現關鍵字次數

In [97]:
# **計算各類別多少篇文章提到該關鍵字
# **計算各類別出現關鍵字次數

news_categories=['全部','焦點','娛樂影劇','國際','政治','社會地方','財經','運動','玩樂','品味','遊戲3C']

def count_keyword(df_query, query_keywords):

    cate_occurrence = {}
    cate_freq = {}
    
    # 字典初始化
    for cate in news_categories:
        cate_occurrence[cate] = 0   # {'政治':0, '科技':0}
        cate_freq[cate] = 0
        

    for idx, row in df_query.iterrows():
        # count the number of articles各類別篇數統計
        cate_occurrence[row.category] += 1  #   {'政治':+1, '科技':0}
        cate_occurrence['全部'] += 1
        
        # count the keyword frequency各類別次數統計
        # 計算這一篇文章的content中重複含有多少個這些關鍵字(頻率)
        freq = sum([ len(re.findall(keyword, row.content, re.I)) for keyword in query_keywords]) 
        cate_freq[row.category] += freq # 在該新聞類別中累計頻率
        cate_freq['全部'] += freq  # 在"全部"類別中累計頻率

    total_articles = cate_occurrence['全部']  # len(df_query)
    total_frequency = cate_freq['全部']
    return cate_freq, cate_occurrence, total_articles, total_frequency


In [98]:
query_keywords = ['川普','美國總統'] 
weeks=4
cond='or'
cate='全部'
df_query = filter_df_via_content(df, query_keywords, cond, cate, weeks)
count_keyword(df_query, query_keywords)

({'全部': 193,
  '焦點': 71,
  '娛樂影劇': 0,
  '國際': 39,
  '政治': 3,
  '社會地方': 9,
  '財經': 58,
  '運動': 5,
  '玩樂': 1,
  '品味': 2,
  '遊戲3C': 5},
 {'全部': 41,
  '焦點': 17,
  '娛樂影劇': 0,
  '國際': 7,
  '政治': 1,
  '社會地方': 2,
  '財經': 8,
  '運動': 2,
  '玩樂': 1,
  '品味': 1,
  '遊戲3C': 2},
 41,
 193)

In [99]:
query_keywords = ['川普','美國總統'] 
weeks=4
cond='or'
cate='全部'
df_query = filter_df_via_content(df, query_keywords, cond, cate, weeks)
count_keyword(df_query, query_keywords)

({'全部': 193,
  '焦點': 71,
  '娛樂影劇': 0,
  '國際': 39,
  '政治': 3,
  '社會地方': 9,
  '財經': 58,
  '運動': 5,
  '玩樂': 1,
  '品味': 2,
  '遊戲3C': 5},
 {'全部': 41,
  '焦點': 17,
  '娛樂影劇': 0,
  '國際': 7,
  '政治': 1,
  '社會地方': 2,
  '財經': 8,
  '運動': 2,
  '玩樂': 1,
  '品味': 1,
  '遊戲3C': 2},
 41,
 193)

In [100]:
import re

In [101]:
content="美國總統川普準備徵收汽車關稅，美股主要指數全數下跌，台積電重挫4.09%，台積電台北現股今日早盤以961元開出後失守960價位，終場收在958元，下跌22元或2.24%；台股以22093點開出後失守22000關卡，收盤以21951點作收，下跌308點。川普已經正式宣布，將對所有不在美國製造的汽車徵收25%的進口稅"
keyword="川普"
re.findall(keyword, content)

['川普', '川普']

In [102]:
len(re.findall(keyword, content))

2

In [103]:
keyword="川普"
re.findall(keyword, content)

['川普', '川普']

In [104]:
keyword="唐納川普"
re.findall(keyword, content)

[]

# Caclulate date-based reported frequency of these keywords計算被報導的次數以時間為基礎

In [105]:
def get_keyword_occurrence_time_series(df_query):
    date_samples = df_query.date
    query_freq = pd.DataFrame({'date_index': pd.to_datetime(date_samples), 'freq': [1 for _ in range(len(df_query))]})
    data = query_freq.groupby(pd.Grouper(key='date_index', freq='D')).sum()
    line_xy_data = []
    for i, idx in enumerate(data.index):
        row = {'x': idx.strftime('%Y-%m-%d'), 'y': int(data.iloc[i].freq)}
        line_xy_data.append(row)
    return line_xy_data

In [106]:
get_keyword_occurrence_time_series(df_query)

[{'x': '2025-03-24', 'y': 1},
 {'x': '2025-03-25', 'y': 0},
 {'x': '2025-03-26', 'y': 1},
 {'x': '2025-03-27', 'y': 39}]

# Put them together

In [107]:
query_keywords = ['川普','美國總統'] 
weeks=4
cond='or'
cate='全部'

df_query = filter_df_via_content(df, query_keywords, cond, cate, weeks)
cate_freq, cate_occurrence, total_articles, total_frequency = count_keyword(df_query, query_keywords)
freqByDate = get_keyword_occurrence_time_series(df_query)


selectedCategories=['全部','焦點','娛樂影劇','國際','政治']
# selectedCategories = ['政治', '產經', '生活', '國際', '社會', '兩岸','全部']

freqByCate = [cate_occurrence[k] for k in selectedCategories]

response =  {'freqByDate': freqByDate,
           'freqByCate': freqByCate,
           'category': selectedCategories,
           'num_frequency': cate_freq['全部'], # 這關鍵字被提多少次
           'num_occurrence': cate_occurrence['全部'] #多少篇提到這關鍵字
           }

In [108]:
freqByCate

[41, 17, 0, 7, 1]

In [109]:
cate_freq

{'全部': 193,
 '焦點': 71,
 '娛樂影劇': 0,
 '國際': 39,
 '政治': 3,
 '社會地方': 9,
 '財經': 58,
 '運動': 5,
 '玩樂': 1,
 '品味': 2,
 '遊戲3C': 5}

In [110]:
cate_occurrence

{'全部': 41,
 '焦點': 17,
 '娛樂影劇': 0,
 '國際': 7,
 '政治': 1,
 '社會地方': 2,
 '財經': 8,
 '運動': 2,
 '玩樂': 1,
 '品味': 1,
 '遊戲3C': 2}

In [111]:
freqByDate

[{'x': '2025-03-24', 'y': 1},
 {'x': '2025-03-25', 'y': 0},
 {'x': '2025-03-26', 'y': 1},
 {'x': '2025-03-27', 'y': 39}]

In [112]:
response

{'freqByDate': [{'x': '2025-03-24', 'y': 1},
  {'x': '2025-03-25', 'y': 0},
  {'x': '2025-03-26', 'y': 1},
  {'x': '2025-03-27', 'y': 39}],
 'freqByCate': [41, 17, 0, 7, 1],
 'category': ['全部', '焦點', '娛樂影劇', '國際', '政治'],
 'num_frequency': 193,
 'num_occurrence': 41}

## Save data to csv file

In [113]:
df_data = pd.DataFrame(list(response.items()),columns=['name','value'])

In [114]:
df_data

,name,value
0,freqByDate,"[{'x': '2025-03-24', 'y': 1}, {'x': '2025-03-2..."
1,freqByCate,"[41, 17, 0, 7, 1]"
2,category,"[全部, 焦點, 娛樂影劇, 國際, 政治]"
3,num_frequency,193
4,num_occurrence,41


In [115]:
## 存成csv格式檔案
df_data.to_csv('chen_shih_chung_data.csv',sep=',', index=None)

### Alternative way: using zip

In [116]:
k=list(response.keys())
v=list(response.values())

In [117]:
#list(zip(k,v))

In [118]:
df_data = pd.DataFrame(list(zip(k,v)),columns=['name','value'])
df_data

,name,value
0,freqByDate,"[{'x': '2025-03-24', 'y': 1}, {'x': '2025-03-2..."
1,freqByCate,"[41, 17, 0, 7, 1]"
2,category,"[全部, 焦點, 娛樂影劇, 國際, 政治]"
3,num_frequency,193
4,num_occurrence,41


## Read csv file and convert to dict format

In [119]:
df_data = pd.read_csv('chen_shih_chung_data.csv')

In [120]:
df_data 

,name,value
0,freqByDate,"[{'x': '2025-03-24', 'y': 1}, {'x': '2025-03-2..."
1,freqByCate,"[41, 17, 0, 7, 1]"
2,category,"['全部', '焦點', '娛樂影劇', '國際', '政治']"
3,num_frequency,193
4,num_occurrence,41


In [121]:
# Convert to dictionary format
dict(list(df_data.values))

{'freqByDate': "[{'x': '2025-03-24', 'y': 1}, {'x': '2025-03-25', 'y': 0}, {'x': '2025-03-26', 'y': 1}, {'x': '2025-03-27', 'y': 39}]",
 'freqByCate': '[41, 17, 0, 7, 1]',
 'category': "['全部', '焦點', '娛樂影劇', '國際', '政治']",
 'num_frequency': '193',
 'num_occurrence': '41'}

In [122]:
list(df_data.values)

[array(['freqByDate',
        "[{'x': '2025-03-24', 'y': 1}, {'x': '2025-03-25', 'y': 0}, {'x': '2025-03-26', 'y': 1}, {'x': '2025-03-27', 'y': 39}]"],
       dtype=object),
 array(['freqByCate', '[41, 17, 0, 7, 1]'], dtype=object),
 array(['category', "['全部', '焦點', '娛樂影劇', '國際', '政治']"], dtype=object),
 array(['num_frequency', '193'], dtype=object),
 array(['num_occurrence', '41'], dtype=object)]

In [123]:
response = dict(list(df_data.values))


In [124]:
response['num_occurrence']

'41'

In [125]:
type(response['num_occurrence'])


str

In [126]:
type(response['freqByCate'])


str

In [127]:
response['freqByCate'] = eval(response['freqByCate'])


In [128]:
response


{'freqByDate': "[{'x': '2025-03-24', 'y': 1}, {'x': '2025-03-25', 'y': 0}, {'x': '2025-03-26', 'y': 1}, {'x': '2025-03-27', 'y': 39}]",
 'freqByCate': [41, 17, 0, 7, 1],
 'category': "['全部', '焦點', '娛樂影劇', '國際', '政治']",
 'num_frequency': '193',
 'num_occurrence': '41'}

### # How to convert a list into a dict?

In [129]:
# How to convert a list into a dict?
[['one',[1,2,3]], ['two',2]]

[['one', [1, 2, 3]], ['two', 2]]

In [130]:
# How to convert a list into a dict?
[['one',[1,2,3]], ['two',2]]

[['one', [1, 2, 3]], ['two', 2]]

In [131]:
dict([['one',[1,2,3]], ['two',2]])

{'one': [1, 2, 3], 'two': 2}

## All-in-one function

In [132]:
# Load Data 
import pandas as pd
from datetime import datetime, timedelta

df = pd.read_csv('./news_dataset_preprocessed_for_django.csv',sep='|')

In [135]:


def process_data_all_in_one(query_keywords, weeks):


    cond='or'
    cate='全部'

    df_query = filter_df_via_content(df, query_keywords, cond, cate, weeks)
    cate_freq, cate_occurrence, _, _ = count_keyword(df_query, query_keywords)
    freqByDate = get_keyword_occurrence_time_series(df_query)


    selectedCategories=['全部','焦點','娛樂影劇','國際','政治','社會地方','財經']
    # selectedCategories = ['全部', '政治', '產經', '生活', '社會']
    # selectedCategories = ['全部', '政治', '兩岸', '產經', '生活', '社會']
    # selectedCategories = ['政治', '產經', '生活', '國際', '社會', '兩岸','全部']

    freqByCate = [cate_occurrence[k] for k in selectedCategories]

    response =  {'freqByDate': freqByDate,
            'freqByCate': freqByCate,
            'category': selectedCategories,
            'num_frequency': cate_freq['全部'], # 這關鍵字被提多少次
            'num_occurrence': cate_occurrence['全部'] #多少篇提到這關鍵字
            }

    

    return response

In [136]:
query_keywords = ['川普','美國總統'] 
weeks=4
data_response = process_data_all_in_one(query_keywords, weeks)

In [139]:
df_data = pd.DataFrame(list(data_response.items()),columns=['name','value'])
## 存成csv格式檔案
df_data.to_csv('Trump_data.csv',sep=',', index=None)

In [140]:
df_data = pd.read_csv('Trump_data.csv')

In [141]:
df_data 

,name,value
0,freqByDate,"[{'x': '2025-03-24', 'y': 1}, {'x': '2025-03-2..."
1,freqByCate,"[41, 17, 0, 7, 1, 2, 8]"
2,category,"['全部', '焦點', '娛樂影劇', '國際', '政治', '社會地方', '財經']"
3,num_frequency,193
4,num_occurrence,41


# This is your featured app

In [144]:
query_keywords = ['柯文哲','柯P'] #不要有子字串否則會重複計算 ['陳時中','時中'] 這樣沒有意義!
#query_keywords = ['柯文哲','柯P','柯p'] #不要有子字串否則會重複計算 
#query_keywords = ['蘋果','Apple','ipad','iphone','imac'] #不要有子字串否則會重複計算 
weeks=4
data_response = process_data_all_in_one(query_keywords, weeks)

In [145]:
data_response

{'freqByDate': [{'x': '2025-03-23', 'y': 1},
  {'x': '2025-03-24', 'y': 1},
  {'x': '2025-03-25', 'y': 1},
  {'x': '2025-03-26', 'y': 2},
  {'x': '2025-03-27', 'y': 8}],
 'freqByCate': [13, 4, 0, 2, 3, 2, 0],
 'category': ['全部', '焦點', '娛樂影劇', '國際', '政治', '社會地方', '財經'],
 'num_frequency': 75,
 'num_occurrence': 13}

In [147]:
query_keywords = ['京華城','北檢署'] #不要有子字串否則會重複計算 ['陳時中','時中'] 這樣沒有意義!
weeks=4
data_response = process_data_all_in_one(query_keywords, weeks)

In [148]:
data_response

{'freqByDate': [{'x': '2025-03-26', 'y': 2}, {'x': '2025-03-27', 'y': 7}],
 'freqByCate': [9, 3, 0, 2, 0, 2, 0],
 'category': ['全部', '焦點', '娛樂影劇', '國際', '政治', '社會地方', '財經'],
 'num_frequency': 11,
 'num_occurrence': 9}

# views.py in Django

In [149]:
from django.http import JsonResponse
from django.shortcuts import render
import pandas as pd

def load_data_scchen():
    # Read data from csv file
    df_data = pd.read_csv('app_scchen/dataset/chen_shih_chung_data.csv',sep=',')
    global response
    response = dict(list(df_data.values))
    del df_data

# load pk data
load_data_scchen()

def home(request):
    return render(request,'app_scchen/home.html', response)

print('app_scchen was loaded!')

FileNotFoundError: [Errno 2] No such file or directory: 'app_scchen/dataset/chen_shih_chung_data.csv'

        # Saving data with other file format

        ## Save to text file
        # Save data
        f = open('pk_politician.txt','w')
        f.write(str(data_pk))
        f.close()

        # Load data
        f = open('pk_politician.txt','r')
        data_pk = f.read()

        # dictionary format
        eval(data_pk)
        ## Save to json file
        import json

        # Save data
        with open('pk_politician.txt', 'w') as file:
            json.dump( data_pk, file)

        # Load data
        with open('pk_politician.txt', 'r') as file:
            data_pk = json.load(file)

        # dictionary format
        data_pk

        ## save to mongoDb (A famous NoneSQL)
        from pymongo import MongoClient

        # Save data
        client = MongoClient()
        database = client["cnaNews"]  # SQL: Database Name
        table_KTH = database["pk_politicianTsaiHan"]   # SQL: Table Name

        table_KTH.drop()
        table_KTH.insert_one(data)


        # Load data
        from pymongo import MongoClient
        client = MongoClient()
        database = client["cnaNews"]  # SQL: Database Name
        table_KTH  = database["pkData"]   # SQL: Table Name

        for x in table_KTH.find():
            print(x)

        list(table_KTH.find({}, {'_id': False}))
